<a href="https://colab.research.google.com/github/KoSeoAh/ML_2022/blob/main/Crawling_Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium==3.141
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 5.0 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/pp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib

options = wd.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [4]:
def get_article_info(driver, crawl_date, press_list, title_list, link_list, date_list, 
                     more_news_base_url=None, more_news=False):
    more_news_url_list = []
    while True:    
        page_html_source = driver.page_source
        url_soup = BeautifulSoup(page_html_source, 'lxml')
        
        more_news_infos = url_soup.select('a.news_more')
        
        if more_news:
            for more_news_info in more_news_infos:
                more_news_url = f"{more_news_base_url}{more_news_info.get('href')}"

                more_news_url_list.append(more_news_url)

        article_infos = url_soup.select("div.news_area")
        
        if not article_infos:
            break

        for article_info in article_infos:
            press_info = article_info.select_one("div.info_group > a.info.press")
            
            if press_info is None:
                press_info = article_info.select_one("div.info_group > span.info.press")
            article = article_info.select_one("a.news_tit")
            
            press = press_info.text.replace("언론사 선정", "")
            title = article.get('title')
            link = article.get('href')

            press_list.append(press)
            title_list.append(title)
            link_list.append(link)
            date_list.append(crawl_date)

        time.sleep(2.0)
                      
                      
        next_button_status = url_soup.select_one("a.btn_next").get("aria-disabled")
        
        if next_button_status == 'true':
            break
        
        time.sleep(1.0)
        next_page_btn = driver.find_element_by_css_selector("a.btn_next").click()      
    
    return press_list, title_list, link_list, more_news_url_list
    
    

def get_naver_news_info_from_selenium(driver_path, keyword, save_path, target_date, ds_de, sort=0, remove_duplicate=False):
    crawl_date = f"{target_date[:4]}.{target_date[4:6]}.{target_date[6:]}"
    driver = wd.Chrome(driver_path, options=options)

    encoded_keyword = urllib.parse.quote(keyword)
    url = f"https://search.naver.com/search.naver?where=news&query={encoded_keyword}&sm=tab_opt&sort={sort}&photo=0&field=0&pd=3&ds={ds_de}&de={ds_de}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{target_date}to{target_date}&is_sug_officeid=0"
    
    more_news_base_url = "https://search.naver.com/search.naver"

    driver.implicitly_wait(0.8)

    driver.get(url)
    
    press_list, title_list, link_list, date_list, more_news_url_list = [], [], [], [], []
    
    press_list, title_list, link_list, more_news_url_list = get_article_info(driver=driver, 
                                                                             crawl_date=crawl_date, 
                                                                             press_list=press_list, 
                                                                             title_list=title_list, 
                                                                             link_list=link_list,
                                                                             date_list=date_list,
                                                                             more_news_base_url=more_news_base_url,
                                                                             more_news=True)
    driver.close()
    
    if len(more_news_url_list) > 0:
        print(len(more_news_url_list))
        more_news_url_list = list(set(more_news_url_list))
        print(f"->{len(more_news_url_list)}")
        for more_news_url in more_news_url_list:
            driver = wd.Chrome(driver_path, options=options)
            driver.implicitly_wait(0.8)
            driver.get(more_news_url)
            
            press_list, title_list, link_list, more_news_url_list = get_article_info(driver=driver, 
                                                                             crawl_date=crawl_date, 
                                                                             press_list=press_list, 
                                                                             title_list=title_list, 
                                                                             link_list=link_list,
                                                                             date_list=date_list)
            driver.close()
    article_df = pd.DataFrame({"날짜": date_list, "언론사": press_list, "제목": title_list, "링크": link_list})
    
    print(f"extract article num : {len(article_df)}")
    if remove_duplicate:
        article_df = article_df.drop_duplicates(['링크'], keep='first')
        print(f"after remove duplicate -> {len(article_df)}")
    
    article_df.to_excel(save_path, index=False)
    

In [5]:
from datetime import datetime
from tqdm import tqdm

def crawl_news_data(driver_path, keyword, year, month, start_day, end_day, save_path_dir):
    for day in tqdm(range(start_day, end_day+1)):
        date_time_obj = datetime(year=year, month=month, day=day)
        target_date = date_time_obj.strftime("%Y%m%d")
        ds_de = date_time_obj.strftime("%Y.%m.%d")

        get_naver_news_info_from_selenium(driver_path=driver_path, keyword=keyword, 
                                          save_path=f"{save_path_dir}/{keyword}/{target_date}_{keyword}_.xlsx", target_date=target_date, ds_de=ds_de, remove_duplicate=False)
        
def bulk_keyword_crawler(driver_path, keywords, year, month, start_day, end_day, save_path_dir):
    for keyword in keywords:
        print(f"start keyword - {keyword} crawling ...")
        crawl_news_data(driver_path=driver_path, keyword=keyword, year=year, month=month, start_day=start_day, end_day=end_day, save_path_dir=save_path_dir)

In [ ]:
import os

year, month = 2017, 5
start_day, end_day = 16, 31
keywords = ['기아', '삼성바이오로직스', '삼성전자',
            '삼성SDI', '카카오', '포스코', 
            '현대차', 'LG화학', 'SK하이닉스']
save_path_dir = "/content/drive/MyDrive/ML"
driver_path = "chromedriver"

#for keyword in keywords:
#  os.makedirs(f"{save_path_dir}/{keyword}")

bulk_keyword_crawler(driver_path=driver_path, keywords=keywords, year=year, month=month, 
                     start_day=start_day, end_day=end_day, save_path_dir=save_path_dir)

start keyword - 기아 crawling ...


  0%|          | 0/16 [00:00<?, ?it/s]

17
->14
extract article num : 658


  6%|▋         | 1/16 [05:39<1:24:54, 339.63s/it]

4
->4


...

extract article num : 80
38
->32


 81%|████████▏ | 13/16 [2:51:11<33:40, 673.44s/it]

extract article num : 1464
82
->71
extract article num : 2985


 88%|████████▊ | 14/16 [3:15:20<30:15, 907.57s/it]

44
->43


In [ ]:
import pandas as pd
import glob

def merge_excel_files(file_path, file_format, save_path, save_format, columns=None):
  merge_df = pd.DataFrame()
  file_list = [f"{file_path}/{file}" for file in os.listdir(file_path) if file_format in file]

  for file in file_list:
    if file_format == ".xlsx":
      file_df = pd.read_excel(file)
    else:
      file_df = pd.read_csv(file)

    if columns is None:
      columns = file_df.columns
    
    temp_df = pd.DataFrame(file_df, columns=columns)

    merge_df = merge_df.append(temp_df)

  if save_format == ".xlsx":
    merge_df.to_excel(save_path, index=False)
  else:
    merge_df.to_csv(save_path, index=False)

if __name__ == "__main__":
  for keyword in keywords:
    merge_excel_files(file_path=f"/content/drive/MyDrive/ML/{keyword}", file_format=".xlsx",
                      save_path=f"/content/drive/MyDrive/ML/{keyword}/000000~000000_{keyword}_merged.xlsx", save_format=".xlsx")